In [6]:
import pandas as pd
from collections import OrderedDict
from datetime import datetime
import numpy as np
import json

import sys
sys.path.append('../python')
import userHistorial as uh
import ELO as elo
import timeLimitEvents as timeLimits

In [7]:
dataEvents = pd.read_csv('E:/Documentos/PCEO/5/Informatica/TFG/datos/anonamyze_all_data_collection_v2.csv', sep=";")
percentilesPuzzles = pd.read_csv('E:/Documentos/PCEO/5/Informatica/TFG/scripts/TFG-Informatica/Outputs/avgTimeByPuzzleOutput.csv', sep=";")
percentilesPuzzles

,Unnamed: 0,puzzle,order,avg_complete_time,TotalTime,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,0,1. One Box,1.0,1.09,0.88,NaN,NaN,NaN,NaN
1,1,2. Separated Boxes,2.0,1.69,1.67,NaN,NaN,NaN,NaN
2,2,3. Rotate a Pyramid,3.0,1.62,1.59,NaN,NaN,NaN,NaN
3,3,4. Match Silhouettes,4.0,2.13,2.12,NaN,NaN,NaN,NaN
4,5,5. Removing Objects,5.0,2.89,2.85,NaN,NaN,NaN,NaN
5,6,6. Stretch a Ramp,6.0,2.43,2.39,NaN,NaN,NaN,NaN
6,7,7. Max 2 Boxes,7.0,1.63,1.53,NaN,NaN,NaN,NaN
7,8,8. Combine 2 Ramps,8.0,2.02,1.92,NaN,NaN,NaN,NaN
8,9,9. Scaling Round Objects,9.0,2.57,2.51,NaN,NaN,NaN,NaN
9,23,Square Cross-Sections,10.0,3.20,3.54,NaN,NaN,NaN,NaN


In [8]:
tiemposPuzzles = dict()
for enum,row in percentilesPuzzles.iterrows():
    tiemposPuzzles[percentilesPuzzles.loc[enum,'puzzle']] = percentilesPuzzles.loc[enum,'avg_complete_time']

#tiemposPuzzles

In [9]:
# Every puzzle difficulty
difficultyMapping = ['Sandbox~0.000001','1. One Box~0.000002', '2. Separated Boxes~0.111127', '3. Rotate a Pyramid~0.083447', '4. Match Silhouettes~0.061887', '5. Removing Objects~0.106021', '6. Stretch a Ramp~0.107035', '7. Max 2 Boxes~0.078039', '8. Combine 2 Ramps~0.068608', '9. Scaling Round Objects~0.128647',
                     'Square Cross-Sections~0.199714', 'Bird Fez~0.156674', 'Pi Henge~0.067346', '45-Degree Rotations~0.096715',  'Pyramids are Strange~0.179600', 'Boxes Obscure Spheres~0.266198', 'Object Limits~0.257177', 'Not Bird~0.260197', 'Angled Silhouette~0.147673',
                     'Warm Up~0.183971','Tetromino~0.226869', 'Stranger Shapes~0.283971', 'Sugar Cones~0.085909', 'Tall and Small~0.266869', 'Ramp Up and Can It~0.206271', 'More Than Meets Your Eye~0.192319', 'Unnecessary~0.76', 'Zzz~0.234035', 'Bull Market~0.358579', 'Few Clues~0.324041', 'Orange Dance~0.647731', 'Bear Market~1.000000']


typeMapping = ['Sandbox~SAND', '1. One Box~Tutorial', '2. Separated Boxes~Tutorial', '3. Rotate a Pyramid~Tutorial', '4. Match Silhouettes~Tutorial', '5. Removing Objects~Tutorial', '6. Stretch a Ramp~Tutorial', '7. Max 2 Boxes~Tutorial', '8. Combine 2 Ramps~Tutorial', '9. Scaling Round Objects~Tutorial',
               'Square Cross-Sections~Easy Puzzles', 'Bird Fez~Easy Puzzles', 'Pi Henge~Easy Puzzles', '45-Degree Rotations~Easy Puzzles',  'Pyramids are Strange~Easy Puzzles', 'Boxes Obscure Spheres~Easy Puzzles', 'Object Limits~Easy Puzzles', 'Not Bird~Easy Puzzles', 'Angled Silhouette~Easy Puzzles',
               'Warm Up~Hard Puzzles','Tetromino~Hard Puzzles', 'Stranger Shapes~Hard Puzzles', 'Sugar Cones~Hard Puzzles', 'Tall and Small~Hard Puzzles', 'Ramp Up and Can It~Hard Puzzles', 'More Than Meets Your Eye~Hard Puzzles', 'Unnecessary~Hard Puzzles', 'Zzz~Hard Puzzles', 'Bull Market~Hard Puzzles', 'Few Clues~Hard Puzzles', 'Orange Dance~Hard Puzzles', 'Bear Market~Hard Puzzles']

# Dictionary with the difficulty of every puzzle
difficultyPuzzles = dict()

for puzzle in difficultyMapping:
    desc = puzzle.split("~")
    difficultyPuzzles[desc[0]] = float(desc[1])


tutorialPuzzles = []

for puzzle in typeMapping:
    desc = puzzle.split("~")
    if(desc[1] == 'Tutorial'):
        tutorialPuzzles.append(desc[0])

advancedPuzzles = []

for puzzle in typeMapping:
    desc = puzzle.split("~")
    if(desc[1] == 'Hard Puzzles'):
        advancedPuzzles.append(desc[0])


intermediatePuzzles = []

for puzzle in typeMapping:
    desc = puzzle.split("~")
    if(desc[1] == 'Easy Puzzles'):
        intermediatePuzzles.append(desc[0])

allPuzzles = []
for puzzle in typeMapping:
    desc = puzzle.split("~")
    allPuzzles.append(desc[0])




def computeFeatures(dataEvents,percentil=0, user='all', group = 'all'):

    #copia
    copia = dataEvents.copy()

    dataEvents['time'] = pd.to_datetime(dataEvents['time'])
    dataEvents = dataEvents.sort_values('time')

    #iterates in the groups and users of the data
    dataEvents['group'] = [json.loads(x)['group'] if 'group' in json.loads(x).keys() else '' for x in
                           dataEvents['data']]
    dataEvents['user'] = [json.loads(x)['user'] if 'user' in json.loads(x).keys() else '' for x in dataEvents['data']]
    dataEvents['task_id'] = [json.loads(x)['task_id'] if 'task_id' in json.loads(x).keys() else '' for x in
                             dataEvents['data']]

    # removing those rows where we dont have a group and a user that is not guest
    dataEvents = dataEvents[
        ((dataEvents['group'] != '') & (dataEvents['user'] != '') & (dataEvents['user'] != 'guest'))]
    dataEvents['group_user_id'] = dataEvents['group'] + '~' + dataEvents['user']
    dataEvents['group_user_task_id'] = dataEvents['group'] + '~' + dataEvents['user'] + '~' + dataEvents['task_id']

    # filtering to only take the user passed as argument
    if(user != 'all'):
        dataEvents = dataEvents[dataEvents['user'].isin(user)]
    if(group != 'all'):
        dataEvents = dataEvents[dataEvents['group'].isin(group)]

    # the data is grouped by the necessary variables
    activity_by_user = dataEvents.groupby(['group_user_id']).agg({'id': 'count',
                                                                  'type': 'nunique'}).reset_index().rename(columns={'id': 'events',
                                                                                        'type': 'different_events'})



    # Data Cleaning
    #dataEvents['time'] = pd.to_datetime(dataEvents['time'])
    dataEvents = dataEvents.sort_values('time')

    typeEvents = ['ws-snapshot', 'ws-paint', 'ws-rotate_view', 'ws-move_shape', 'ws-rotate_shape', 'ws-scale_shape',
                  'ws-create_shape', 'ws-delete_shape', 'ws-undo_action', 'ws-redo_action', 'ws-check_solution']
    manipulationTypeEvents = ['ws-move_shape', 'ws-rotate_shape', 'ws-scale_shape', 'ws-create_shape',
                              'ws-delete_shape']

    #initialize the metrics
    activity_by_user['completed'] = np.nan
    activity_by_user['active_time'] = np.nan
    activity_by_user['n_events'] = np.nan
    activity_by_user['timestamp'] = np.nan

    for event in typeEvents:
        activity_by_user[event] = 0
#----------------------------------------------------------------------------------------------------------------------------------
    #initialize the data structures

    puzzleEventsPercentile1 = dict() #Diccionario que guarda para cada intento cuantos evenos que ocurren dentro del percentil perdido
    puzzleEventsPercentile2 = dict() #Diccionario que guarda para cada intento cuantos evenos que ocurren dentro del percentil perdido
    puzzleEventsPercentile3 = dict() #Diccionario que guarda para cada intento cuantos evenos que ocurren dentro del percentil perdido

    tiempoPercentil1,tiempoPercentil2,tiempoPercentil3 = [0,0,0] #variable donde guardar el tiempo percentil pedido

    puzzleEvents = dict() #Diccionario que guarda para cada intento (alumno,puzzle,nintento) el número de eventos

    timePuzzle = dict() #Diccionario que guarda para cada intento el tiempo activo

    globalTypesEvents = dict() #Diccionario que guarda para cada intento el número de eventos de cada tipo

    n_attempts = dict() #Diccionario que guarda para cada el número de intentos

    completados = dict() #Diccionario que guarda para cada intento si se completó

    timestamp = dict() #Diccionario que guarda para cada intento el timestamp inicial

    percentilAtt = dict()
    percentilTime = dict() #Diccionarios que guardan un 90 para cada intento (creo)
    percentilAttValue = 90
    percentilTimeValue = 90

    breaksPuzzle = dict() #Diccionario que guarda para cada intento el número de breaks ( parones de 15 segundos )

    cumAttempts = OrderedDict() #Diccionario que guarda para cada intento, que número de intento gloabal es


    userCumAttempts = OrderedDict() #Diccionario que guarda para cada alumno cuantos intentos en total ha hecho


    prevReg = dict() #Diccionario que almacena para cada dupla alumno-puzzle, si se ha intentado anteriormente

    actualAtt = 0 #Variable global que lleva el contador de intentos global

    idComplete = dict() #Diccionario que guarda para cada intento si se finalizó

    attemptsAux = dict() #Doble diccionario que guarda para cada usuario y para cada puzzle el numero de intentos

    contCheckSol = dict() #Diccionario que guarda para cada intento el número de submits

    manipulationEvents = dict() #Diccionario que guarda para cada intento el número de eventos de manipulacion

    userManipulationEvents = dict() #Creo que actualmente no se utiliza

    contManipulation = 0

    timeFirstCheck = dict() #Diccionario que guarda para cada intento el timestamp del primer submit

    timeSubExit = dict() #Diccionario que guarda para cada intento, el tiempo que ha pasado entre el primer submit y el exit

    timeCheckActual = dict() #Diccionario que guarda para cada intento el timestamp del último check

    timeBetweenSub = dict() #Diccionario que guarda para cada intento la media de tiempo entre cada submit
#---------------------------------------------------------------------------------------------------------------------------------
    for user in dataEvents['group_user_id'].unique():

        # Computing active time
        previousEvent = None
        theresHoldActivity = 60

        user_events = dataEvents[dataEvents['group_user_id'] == user] #eventos del usuario
        user_puzzle_key = None
        userParc = None
        task_id = None
        initialTime = None
        prev_id = 1

        for enum, event in user_events.iterrows():

            # If it is the first event
            if (previousEvent is None):
                previousEvent = event
                continue

            if (event['type'] in ['ws-start_level']):

                #create id: group+user+task_id
                task_id = json.loads(event['data'])['task_id']

                if (user_puzzle_key not in timeSubExit.keys()):
                    timeSubExit[user_puzzle_key] = str(0)
                    timeBetweenSub[user_puzzle_key] = str(0)

                if (event['user'] not in userCumAttempts.keys()):
                    userCumAttempts[event['user']] = 0
                    actualAtt = 0
                    attemptsAux[event['user']] = dict()
                    timeCheckActual[event['user']] = 0

                if (event['user'] not in userManipulationEvents.keys()):
                    #print("Se inicializa con ", event['user'])
                    userManipulationEvents[event['user']] = 0

                #if(user_puzzle_key not in manipulationEvents.keys()):
                #    manipulationEvents[user_puzzle_key] = 0
                #    contManipulation = 0

                if (task_id not in attemptsAux[event['user']].keys()): attemptsAux[event['user']][task_id] = 0

                user_puzzle_key = event['group'] + '~' + event[
                    'user'] + '~' + task_id  # + '~' + str(n_attempts[prev_id])
                if (user_puzzle_key not in prevReg.keys()):

                    prevReg[user_puzzle_key] = 1
                    user_puzzle_key = event['group'] + '~' + event['user'] + '~' + task_id + '~' + '1'
                    n_attempts[user_puzzle_key] = 1
                    attemptsAux[event['user']][task_id] = n_attempts[user_puzzle_key]

                else:

                    user_puzzle_key = event['group'] + '~' + event['user'] + '~' + task_id + '~' + str(
                        attemptsAux[event['user']][task_id])
                    n_attempts[user_puzzle_key] = attemptsAux[event['user']][task_id]

                key_split = user_puzzle_key.split('~')
                puzzleParc = key_split[2]
                userParc = key_split[1]

                tiemposPuzzle = percentilesPuzzles[percentilesPuzzles['puzzle']==puzzleParc].reset_index()
                if not tiemposPuzzle.empty:
                    tiempoPercentil1 = round(tiemposPuzzle.loc[0,'avg_complete_time']*0.25,2)
                    tiempoPercentil2 = round(tiemposPuzzle.loc[0,'avg_complete_time']*0.50,2)
                    tiempoPercentil3 = round(tiemposPuzzle.loc[0,'avg_complete_time']*0.75,2)

                if (user_puzzle_key not in idComplete.keys()): idComplete[user_puzzle_key] = 0

                if (task_id not in attemptsAux[userParc].keys()): attemptsAux[userParc][task_id] = 0
                if (user_puzzle_key not in cumAttempts.keys()): cumAttempts[user_puzzle_key] = 1

                # initialize if the id is new
                if (user_puzzle_key not in puzzleEvents.keys()):

                    breaksPuzzle[user_puzzle_key] = 0
                    timestamp[user_puzzle_key] = 0
                    percentilAtt[user_puzzle_key] = percentilAttValue
                    percentilTime[user_puzzle_key] = percentilTimeValue
                    completados[user_puzzle_key] = 0
                    puzzleEvents[user_puzzle_key] = 1
                    puzzleEventsPercentile1[user_puzzle_key] = 1
                    puzzleEventsPercentile2[user_puzzle_key] = 1
                    puzzleEventsPercentile3[user_puzzle_key] = 1
                    timePuzzle[user_puzzle_key] = 0
                    contCheckSol[user_puzzle_key] = 0
                    manipulationEvents[user_puzzle_key] = 0
                    timeFirstCheck[user_puzzle_key] = 0

                    globalTypesEvents[user_puzzle_key] = dict()
                    for ev in typeEvents:
                        globalTypesEvents[user_puzzle_key][ev] = 0

                #timestamp
                if (event['type'] in 'ws-start_level'):
                    timestamp[user_puzzle_key] = event['time']
                    initialTime = timestamp[user_puzzle_key]

            # the event is not final event
            if (event['type'] not in ['ws-exit_to_menu', 'ws-disconnect', 'ws-create_user', 'ws-login_user']):

                if (event['type'] in ['ws-puzzle_complete']): completados[user_puzzle_key] = 1

                puzzleEvents[user_puzzle_key] += 1

                if round((timePuzzle[user_puzzle_key]/60),2) < tiempoPercentil1:
                    puzzleEventsPercentile1[user_puzzle_key] += 1

                if round((timePuzzle[user_puzzle_key]/60),2) < tiempoPercentil2:
                    puzzleEventsPercentile2[user_puzzle_key] += 1

                if round((timePuzzle[user_puzzle_key]/60),2) < tiempoPercentil3:
                    puzzleEventsPercentile3[user_puzzle_key] += 1

                #calculate the duration of the event
                delta_seconds = (event['time'] - previousEvent['time']).total_seconds()
                if ((delta_seconds < theresHoldActivity)):
                    timePuzzle[user_puzzle_key] += delta_seconds

                #breaks
                if ((delta_seconds > 15)):
                    breaksPuzzle[user_puzzle_key] += 1

                previousEvent = event

                #update event counters by type
                if (event['type'] in typeEvents):
                    globalTypesEvents[user_puzzle_key][event['type']] += 1

                if (globalTypesEvents[user_puzzle_key]['ws-check_solution'] == 1): timeFirstCheck[user_puzzle_key] = \
                    event['time']

                if (event['type'] in manipulationTypeEvents):
                    manipulationEvents[user_puzzle_key] += 1

                if (event['type'] == 'ws-check_solution'):
                    timeCheckActual[event['user']] = event['time']
                    contCheckSol[user_puzzle_key] += 1

            # the puzzle ends
            if (event['type'] in ['ws-exit_to_menu', 'ws-disconnect']):

                idComplete[user_puzzle_key] = 1
                puzzleEvents[user_puzzle_key] += 1

                if round((timePuzzle[user_puzzle_key]/60),2) < tiempoPercentil1:
                    puzzleEventsPercentile1[user_puzzle_key] += 1

                if round((timePuzzle[user_puzzle_key]/60),2) < tiempoPercentil2:
                    puzzleEventsPercentile2[user_puzzle_key] += 1

                if round((timePuzzle[user_puzzle_key]/60),2) < tiempoPercentil3:
                    puzzleEventsPercentile3[user_puzzle_key] += 1

                if (completados[user_puzzle_key] == 0 and globalTypesEvents[user_puzzle_key]['ws-check_solution'] > 0):
                    timeSubExit[user_puzzle_key] = str(
                        round((event['time'] - timeFirstCheck[user_puzzle_key]).total_seconds(), 2))
                else:
                    timeSubExit[user_puzzle_key] = 'NA'

                if (globalTypesEvents[user_puzzle_key]['ws-check_solution'] == 0):
                    timeBetweenSub[user_puzzle_key] = 'NA'
                else:
                    timeBetweenSub[user_puzzle_key] = str(round(((timeCheckActual[event['user']] - timestamp[
                        user_puzzle_key]) / globalTypesEvents[user_puzzle_key]['ws-check_solution']).total_seconds(),
                                                                2))

                #calculate the duration of the event
                delta_seconds = (event['time'] - previousEvent['time']).total_seconds()
                if ((delta_seconds < theresHoldActivity)):
                    timePuzzle[user_puzzle_key] += delta_seconds

                #breaks
                if ((delta_seconds > 15)):
                    breaksPuzzle[user_puzzle_key] += 1

                previousEvent = event

                userCumAttempts[userParc] += 1
                n_attempts[user_puzzle_key] += 1
                actualAtt += 1
                cumAttempts[user_puzzle_key] = actualAtt
                attemptsAux[userParc][task_id] = n_attempts[user_puzzle_key]

                #manipulationEvents[user_puzzle_key] = userManipulationEvents[event['user']]

                ###########

    userTime = dict()
    userAtt = dict()
    userEvent = dict()


    for i in puzzleEvents.keys():
        if(idComplete[i]==0):
            continue
        key_split = i.split('~')
        if(key_split[1] not in userTime.keys()):
            userTime[i] = 0
            userAtt[i] = 0
            userEvent[i] = 0

        if(key_split[2] != ''):

            if(key_split[2] in allPuzzles): userAtt[i] = contCheckSol[i]

            if(key_split[2] in allPuzzles): userTime[i] = timePuzzle[i]

            if(key_split[2] in allPuzzles): userEvent[i] = puzzleEvents[i]


    puzzleTime = dict()
    puzzleAtt = dict()
    puzzleEvent = dict()

    for i in userTime.keys():
        #for puzzle in userTime[user]:
        key_split = i.split('~')
        if(key_split[2] not in puzzleTime.keys()):
            puzzleTime[key_split[2]] = []
            puzzleAtt[key_split[2]] = []
            puzzleEvent[key_split[2]] = []

        puzzleTime[key_split[2]].append(userTime[i])
        puzzleAtt[key_split[2]].append(userAtt[i])
        puzzleEvent[key_split[2]].append(userEvent[i])

    difficultyNumber = dict()

    # Creacion de

    for i in puzzleEvents.keys():
        if(idComplete[i]==0):
            continue

        key_split = i.split('~')

        # recuperamos el historial del usuario
        lista_grupo = []
        lista_grupo.append(key_split[0])
        lista_usuario = []
        lista_usuario.append(key_split[1])

        data_events_aux = timeLimits.computeTimeLimitEvents(copia, timeLimit= timestamp[i])
        user_historial = uh.computeUserHistorial(data_events_aux,group = lista_grupo, user = lista_usuario)

        # recuperamos elo del jugador y dificultad ELO del puzzle
        print('timestamp:' + str(timestamp[i]))
        totalData, train_set, test_set = elo.adaptedData(data_events_aux)
        print('ejecuto run')
        user_elo, puzzle_elo = elo.run(1.8, 0.05, 'standard', totalData, train_set, test_set, user_objective =key_split[1], puzzle_objective=key_split[2], group_objective = key_split[0])
        print('elo: ' + str(user_elo) + ' puzzle: ' + str(puzzle_elo))

        #puzzle difficulty
        difficultyNumber[i] = difficultyPuzzles[key_split[2]]

        if(key_split[2] != '' and key_split[1] != '' and i != ''):

            # User, group and puzzle
            activity_by_user.at[i, 'group'] = key_split[0] # Grupo
            activity_by_user.at[i, 'user'] = key_split[1] # Usuario
            activity_by_user.at[i, 'task_id'] = key_split[2] # Puzzle
            activity_by_user.at[i, 'cum_this_puzzle_attempt'] = key_split[3] # Numero de intento de este usuario en este puzzle
            activity_by_user.at[i, 'timestamp'] = timestamp[i] # Timestamp inicial del intento

            # User historial. Percentage of tutorial,intermediate and advanced per puzzle. Attempts per puzzle
            activity_by_user.at[i, 'percentage_tutorial'] = user_historial['percentage_tutorial']
            activity_by_user.at[i, 'percentage_intermediate'] = user_historial['percentage_intermediate']
            activity_by_user.at[i, 'percentage_advanced'] = user_historial['percentage_advanced']
            activity_by_user.at[i, 'attempts_per_puzzle'] = user_historial['attempts_per_puzzle']

            # User and puzzle ELO
            activity_by_user.at[i, 'user_elo'] = user_elo
            activity_by_user.at[i, 'puzzle_elo'] = puzzle_elo

            # Puzzle difficulty
            activity_by_user.at[i, 'puzzle_difficulty'] = difficultyNumber[i]
            # Avg time completion of the puzzle
            if (key_split[2]=='Sandbox' or key_split[2]=='Tetromino'):
                activity_by_user.at[i, 'avg_complete_time'] = 0
            else:
                activity_by_user.at[i, 'avg_complete_time'] = tiemposPuzzles[key_split[2]]

            activity_by_user.at[i, 'n_events'] = puzzleEvents[i] #numero de eventos

            activity_by_user.at[i, 'n_events_percentile1'] = puzzleEventsPercentile1[i] #numero de eventos antes del primer percentil
            activity_by_user.at[i, 'n_events_percentile2'] = puzzleEventsPercentile2[i] #numero de eventos antes del segundo percentil
            activity_by_user.at[i, 'n_events_percentile3'] = puzzleEventsPercentile3[i] #numero de eventos antes del tercer percentil

            activity_by_user.at[i, 'completed'] = completados[i] #Ha completado el puzzle o no

            activity_by_user.at[i, 'n_breaks'] = breaksPuzzle[i] #numero de breaks en el intento
            activity_by_user.at[i, 'n_manipulation_events'] = manipulationEvents[i] #numero de intentos de manipulacion


            activity_by_user.at[i, 'time_failed_submission_exit'] = timeSubExit[i] #tiempo entre submit y exit
            activity_by_user.at[i, 'avg_time_between_submissions'] = timeBetweenSub[i] #tiempo entre submits
            activity_by_user.at[i, 'n_check_solution'] = globalTypesEvents[i]['ws-check_solution'] #numero de submits
            activity_by_user.at[i, 'n_snapshot'] = globalTypesEvents[i]['ws-snapshot']  #numero de snaphshots
            activity_by_user.at[i, 'n_rotate_view'] = globalTypesEvents[i]['ws-rotate_view'] #numero de rotaciones

    #delete row with NaN
    activity_by_user.dropna(subset = ['user'], inplace=True)


    #data output preparation
    activity_by_user = pd.DataFrame(activity_by_user, columns=['group', 'user','task_id','cum_this_puzzle_attempt','timestamp','percentage_tutorial','percentage_intermediate','percentage_advanced','attempts_per_puzzle','user_elo','puzzle_elo','puzzle_difficulty','avg_complete_time','completed', 'active_time','n_events','n_events_percentile1','n_events_percentile2','n_events_percentile3'])

    return activity_by_user

In [ ]:
pruebas = computeFeatures(dataEvents, user = ['0d35fa2868a21199e36b159b49ae6aad'])

timestamp:2019-10-23 13:10:05.094066-04:00
ejecuto run
El elo es : 0.0
elo: 0.0 puzzle: 0.0
timestamp:2019-10-23 13:12:03.911763-04:00
ejecuto run
El elo es : 0.0
elo: 0.0 puzzle: 0.14
timestamp:2019-10-23 13:13:21.359373-04:00
ejecuto run
El elo es : 0.0
elo: 0.0 puzzle: 0.29
timestamp:2019-10-23 13:14:16.791967-04:00
ejecuto run
El elo es : 0.0
elo: 0.0 puzzle: 0.51
timestamp:2019-10-23 13:16:40.642590-04:00
ejecuto run
El elo es : 0.0
elo: 0.0 puzzle: 0.59
timestamp:2019-10-23 13:17:17.630314-04:00
ejecuto run
El elo es : 0.0
elo: 0.0 puzzle: 0.53
timestamp:2019-10-23 13:17:57.747873-04:00
ejecuto run
El elo es : 0.0
elo: 0.0 puzzle: 0.72
timestamp:2019-10-23 13:19:14.516245-04:00
ejecuto run
El elo es : 0.0
elo: 0.0 puzzle: 0.71
timestamp:2019-10-23 13:20:05.965043-04:00
ejecuto run
El elo es : 0.0
elo: 0.0 puzzle: 0.71
timestamp:2019-10-23 13:20:52.059524-04:00
ejecuto run
El elo es : 0.0
elo: 0.0 puzzle: 1.24
timestamp:2019-10-23 13:23:42.745469-04:00
ejecuto run
El elo es : 0.18

In [ ]:
pruebas.to_csv("../Outputs/featuresOutput3.csv", decimal=".", sep=";", mode='w')